# Dataset Merging

### Importing libraries

In [1]:
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install seaborn
!pip install scipy

!pip install xlrd
!pip install openpyxl

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
%matplotlib inline

### Importing the data

In [37]:
DATA_PATH = "../data/"

path = DATA_PATH + "Telco_customer_churn.xlsx"

#### Importing the data sheets

We import every sheet and look at the first few data points.

#### Original

In [38]:
df_orig = pd.read_excel(path, sheet_name='Telco_Churn')
df_orig.head()

,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,...,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Score,CLTV,Churn Reason
0,3668-QPYBK,1,United States,California,Los Angeles,90003,"33.964131, -118.272783",33.964131,-118.272783,Male,...,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1,86,3239,Competitor made better offer
1,9237-HQITU,1,United States,California,Los Angeles,90005,"34.059281, -118.30742",34.059281,-118.307420,Female,...,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,1,67,2701,Moved
2,9305-CDSKC,1,United States,California,Los Angeles,90006,"34.048013, -118.293953",34.048013,-118.293953,Female,...,Month-to-month,Yes,Electronic check,99.65,820.5,Yes,1,86,5372,Moved
3,7892-POOKP,1,United States,California,Los Angeles,90010,"34.062125, -118.315709",34.062125,-118.315709,Female,...,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes,1,84,5003,Moved
4,0280-XJGEX,1,United States,California,Los Angeles,90015,"34.039224, -118.266293",34.039224,-118.266293,Male,...,Month-to-month,Yes,Bank transfer (automatic),103.70,5036.3,Yes,1,89,5340,Competitor had better devices


In [39]:
df_orig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 33 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CustomerID         7043 non-null   object 
 1   Count              7043 non-null   int64  
 2   Country            7043 non-null   object 
 3   State              7043 non-null   object 
 4   City               7043 non-null   object 
 5   Zip Code           7043 non-null   int64  
 6   Lat Long           7043 non-null   object 
 7   Latitude           7043 non-null   float64
 8   Longitude          7043 non-null   float64
 9   Gender             7043 non-null   object 
 10  Senior Citizen     7043 non-null   object 
 11  Partner            7043 non-null   object 
 12  Dependents         7043 non-null   object 
 13  Tenure Months      7043 non-null   int64  
 14  Phone Service      7043 non-null   object 
 15  Multiple Lines     7043 non-null   object 
 16  Internet Service   7043 

#### Status

In [40]:
df_status = pd.read_excel(path, sheet_name="status")
df_status.head()

,Customer ID,Count,Quarter,Satisfaction Score,Customer Status,Churn Label,Churn Value,Churn Score,CLTV,Churn Category,Churn Reason
0,8779-QRDMV,1,Q3,3,Churned,Yes,1,91,5433,Competitor,Competitor offered more data
1,7495-OOKFY,1,Q3,3,Churned,Yes,1,69,5302,Competitor,Competitor made better offer
2,1658-BYGOY,1,Q3,2,Churned,Yes,1,81,3179,Competitor,Competitor made better offer
3,4598-XLKNJ,1,Q3,2,Churned,Yes,1,88,5337,Dissatisfaction,Limited range of services
4,4846-WHAFZ,1,Q3,2,Churned,Yes,1,67,2793,Price,Extra data charges


In [41]:
df_status.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Customer ID         7043 non-null   object
 1   Count               7043 non-null   int64 
 2   Quarter             7043 non-null   object
 3   Satisfaction Score  7043 non-null   int64 
 4   Customer Status     7043 non-null   object
 5   Churn Label         7043 non-null   object
 6   Churn Value         7043 non-null   int64 
 7   Churn Score         7043 non-null   int64 
 8   CLTV                7043 non-null   int64 
 9   Churn Category      1869 non-null   object
 10  Churn Reason        1869 non-null   object
dtypes: int64(5), object(6)
memory usage: 605.4+ KB


#### Services

In [42]:
df_services = pd.read_excel(path, sheet_name="services")
df_services.head()

,Customer ID,Count,Quarter,Referred a Friend,Number of Referrals,Tenure in Months,Offer,Phone Service,Avg Monthly Long Distance Charges,Multiple Lines,...,Unlimited Data,Contract,Paperless Billing,Payment Method,Monthly Charge,Total Charges,Total Refunds,Total Extra Data Charges,Total Long Distance Charges,Total Revenue
0,8779-QRDMV,1,Q3,No,0,1,None,No,0.00,No,...,No,Month-to-Month,Yes,Bank Withdrawal,39.65,39.65,0.00,20,0.00,59.65
1,7495-OOKFY,1,Q3,Yes,1,8,Offer E,Yes,48.85,Yes,...,Yes,Month-to-Month,Yes,Credit Card,80.65,633.30,0.00,0,390.80,1024.10
2,1658-BYGOY,1,Q3,No,0,18,Offer D,Yes,11.33,Yes,...,Yes,Month-to-Month,Yes,Bank Withdrawal,95.45,1752.55,45.61,0,203.94,1910.88
3,4598-XLKNJ,1,Q3,Yes,1,25,Offer C,Yes,19.76,No,...,Yes,Month-to-Month,Yes,Bank Withdrawal,98.50,2514.50,13.43,0,494.00,2995.07
4,4846-WHAFZ,1,Q3,Yes,1,37,Offer C,Yes,6.33,Yes,...,Yes,Month-to-Month,Yes,Bank Withdrawal,76.50,2868.15,0.00,0,234.21,3102.36


In [43]:
df_services.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 30 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Customer ID                        7043 non-null   object 
 1   Count                              7043 non-null   int64  
 2   Quarter                            7043 non-null   object 
 3   Referred a Friend                  7043 non-null   object 
 4   Number of Referrals                7043 non-null   int64  
 5   Tenure in Months                   7043 non-null   int64  
 6   Offer                              7043 non-null   object 
 7   Phone Service                      7043 non-null   object 
 8   Avg Monthly Long Distance Charges  7043 non-null   float64
 9   Multiple Lines                     7043 non-null   object 
 10  Internet Service                   7043 non-null   object 
 11  Internet Type                      7043 non-null   objec

#### Location

In [44]:
df_location = pd.read_excel(path, sheet_name="location")
df_location.head()

,Customer ID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude
0,8779-QRDMV,1,United States,California,Los Angeles,90022,"34.02381, -118.156582",34.023810,-118.156582
1,7495-OOKFY,1,United States,California,Los Angeles,90063,"34.044271, -118.185237",34.044271,-118.185237
2,1658-BYGOY,1,United States,California,Los Angeles,90065,"34.108833, -118.229715",34.108833,-118.229715
3,4598-XLKNJ,1,United States,California,Inglewood,90303,"33.936291, -118.332639",33.936291,-118.332639
4,4846-WHAFZ,1,United States,California,Whittier,90602,"33.972119, -118.020188",33.972119,-118.020188


In [45]:
df_location.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Customer ID  7043 non-null   object 
 1   Count        7043 non-null   int64  
 2   Country      7043 non-null   object 
 3   State        7043 non-null   object 
 4   City         7043 non-null   object 
 5   Zip Code     7043 non-null   int64  
 6   Lat Long     7043 non-null   object 
 7   Latitude     7043 non-null   float64
 8   Longitude    7043 non-null   float64
dtypes: float64(2), int64(2), object(5)
memory usage: 495.3+ KB


#### Population

In [46]:
df_population = pd.read_excel(path, sheet_name="population")
df_population.head()

,ID,Zip Code,Population,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29
0,1,90001,54492,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,90002,44586,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,90003,58198,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,90004,67852,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,90005,43019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
df_population.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1671 entries, 0 to 1670
Data columns (total 30 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ID           1671 non-null   int64  
 1   Zip Code     1671 non-null   int64  
 2   Population   1671 non-null   int64  
 3   Unnamed: 3   0 non-null      float64
 4   Unnamed: 4   0 non-null      float64
 5   Unnamed: 5   0 non-null      float64
 6   Unnamed: 6   0 non-null      float64
 7   Unnamed: 7   0 non-null      float64
 8   Unnamed: 8   0 non-null      float64
 9   Unnamed: 9   0 non-null      float64
 10  Unnamed: 10  0 non-null      float64
 11  Unnamed: 11  0 non-null      float64
 12  Unnamed: 12  0 non-null      float64
 13  Unnamed: 13  0 non-null      float64
 14  Unnamed: 14  0 non-null      float64
 15  Unnamed: 15  0 non-null      float64
 16  Unnamed: 16  0 non-null      float64
 17  Unnamed: 17  0 non-null      float64
 18  Unnamed: 18  0 non-null      float64
 19  Unname

#### Demographics

In [48]:
df_demographics = pd.read_excel(path, sheet_name="demographics")
df_demographics.head()

,Customer ID,Count,Gender,Age,Under 30,Senior Citizen,Married,Dependents,Number of Dependents
0,8779-QRDMV,1,Male,78,No,Yes,No,No,0
1,7495-OOKFY,1,Female,74,No,Yes,Yes,Yes,1
2,1658-BYGOY,1,Male,71,No,Yes,No,Yes,3
3,4598-XLKNJ,1,Female,78,No,Yes,Yes,Yes,1
4,4846-WHAFZ,1,Female,80,No,Yes,Yes,Yes,1


In [49]:
df_demographics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Customer ID           7043 non-null   object
 1   Count                 7043 non-null   int64 
 2   Gender                7043 non-null   object
 3   Age                   7043 non-null   int64 
 4   Under 30              7043 non-null   object
 5   Senior Citizen        7043 non-null   object
 6   Married               7043 non-null   object
 7   Dependents            7043 non-null   object
 8   Number of Dependents  7043 non-null   int64 
dtypes: int64(3), object(6)
memory usage: 495.3+ KB


### Concatenation

Let us concatenate the dataframes into a single dataframe as it will be easier to work with. However, we must be careful combining these datasets as we must ensure that the data attributes correspond to the same clients. We also remark that the population dataframe is different from the others as each row does not correspond to a client but an area and an agglomeration of clients. We will see later if/how we can incorporate this into our data. 

Let us see for the other dataframes (excluding population) whether the customer Id's match in every row so that we can merge the data together. We see that not every dataframe has the same label for Customer ID so we first update them. We also see that all the contents of the original dataframe in contained within the four others (status, services, location and demographics).

In [50]:
df_orig.rename(columns={'CustomerID': 'Customer ID'}, inplace=True)

In [51]:
def checkID(dataframes):
    assert len(set(len(dataframe) for dataframe in dataframes)) == 1
    for i in range(len(dataframes[0])):
        for j in range(1, len(dataframes)):
            try:
                assert dataframes[0].iloc[i]["Customer ID"] == dataframes[j].iloc[i]["Customer ID"]
            except:
                print(i, j)
    return "Customer Id's match!"

dataframes = [df_status, df_services, df_location, df_demographics]
checkID(dataframes)

"Customer Id's match!"

We see that these four datasets were indeed designed and created together. Therefore, we can safely concatenate them. 

In [52]:
df = pd.concat([df_status, df_services, df_location, df_demographics], axis=1, join='outer', ignore_index=False, verify_integrity=False)
df.head()

,Customer ID,Count,Quarter,Satisfaction Score,Customer Status,Churn Label,Churn Value,Churn Score,CLTV,Churn Category,...,Longitude,Customer ID,Count,Gender,Age,Under 30,Senior Citizen,Married,Dependents,Number of Dependents
0,8779-QRDMV,1,Q3,3,Churned,Yes,1,91,5433,Competitor,...,-118.156582,8779-QRDMV,1,Male,78,No,Yes,No,No,0
1,7495-OOKFY,1,Q3,3,Churned,Yes,1,69,5302,Competitor,...,-118.185237,7495-OOKFY,1,Female,74,No,Yes,Yes,Yes,1
2,1658-BYGOY,1,Q3,2,Churned,Yes,1,81,3179,Competitor,...,-118.229715,1658-BYGOY,1,Male,71,No,Yes,No,Yes,3
3,4598-XLKNJ,1,Q3,2,Churned,Yes,1,88,5337,Dissatisfaction,...,-118.332639,4598-XLKNJ,1,Female,78,No,Yes,Yes,Yes,1
4,4846-WHAFZ,1,Q3,2,Churned,Yes,1,67,2793,Price,...,-118.020188,4846-WHAFZ,1,Female,80,No,Yes,Yes,Yes,1


If we set `verify_integrity=True`, we'll find that we have overlapping columns: `Customer ID, Count, Quarter`. Therefore, we remove these duplicate columns from our new dataframe. 

In [53]:
df = df.loc[:,~df.columns.duplicated()] # Removes duplicates
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 52 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Customer ID                        7043 non-null   object 
 1   Count                              7043 non-null   int64  
 2   Quarter                            7043 non-null   object 
 3   Satisfaction Score                 7043 non-null   int64  
 4   Customer Status                    7043 non-null   object 
 5   Churn Label                        7043 non-null   object 
 6   Churn Value                        7043 non-null   int64  
 7   Churn Score                        7043 non-null   int64  
 8   CLTV                               7043 non-null   int64  
 9   Churn Category                     1869 non-null   object 
 10  Churn Reason                       1869 non-null   object 
 11  Referred a Friend                  7043 non-null   objec

Having removed duplicates, we now have 51 variables. 

We save the new dataset into a new file so that we do not have to rerun the code every time. 

In [54]:
df.isnull().sum(axis=0)

Customer ID                             0
Count                                   0
Quarter                                 0
Satisfaction Score                      0
Customer Status                         0
Churn Label                             0
Churn Value                             0
Churn Score                             0
CLTV                                    0
Churn Category                       5174
Churn Reason                         5174
Referred a Friend                       0
Number of Referrals                     0
Tenure in Months                        0
Offer                                   0
Phone Service                           0
Avg Monthly Long Distance Charges       0
Multiple Lines                          0
Internet Service                        0
Internet Type                           0
Avg Monthly GB Download                 0
Online Security                         0
Online Backup                           0
Device Protection Plan            

In [55]:
df.groupby("Churn Category")["Customer ID"].nunique()

Churn Category
Attitude           314
Competitor         841
Dissatisfaction    303
Other              200
Price              211
Name: Customer ID, dtype: int64

In [56]:
df.groupby("Churn Reason")["Customer ID"].nunique()

Churn Reason
Attitude of service provider                  94
Attitude of support person                   220
Competitor had better devices                313
Competitor made better offer                 311
Competitor offered higher download speeds    100
Competitor offered more data                 117
Deceased                                       6
Don't know                                   130
Extra data charges                            39
Lack of affordable download/upload speed      30
Lack of self-service on Website               29
Limited range of services                     37
Long distance charges                         64
Moved                                         46
Network reliability                           72
Poor expertise of online support              31
Poor expertise of phone support               12
Price too high                                78
Product dissatisfaction                       77
Service dissatisfaction                       63
Name: C

In [57]:
df.groupby("Churn Label")["Customer ID"].nunique()

Churn Label
No     5174
Yes    1869
Name: Customer ID, dtype: int64

In [58]:
df.groupby("Customer Status")["Customer ID"].nunique()

Customer Status
Churned    1869
Joined      454
Stayed     4720
Name: Customer ID, dtype: int64

In [59]:
df.groupby("Churn Value")["Customer ID"].nunique()

Churn Value
0    5174
1    1869
Name: Customer ID, dtype: int64

In [60]:
4719 + 453

5172

Remarks: Clearly the customers with the status of "Joined" have been considered as customers who are not going to churn. Decision to be made: Include or not include? We can try both. I propose leaving it for now and then trying running our algorithms at the end of the project. There is an interesting tradeoff. We would expect that removing the 450 or so customers who have only "joined" would make the features of customers who do not churn more precise and increase their importance, improving accuracy. On the other hand we are removing training instances so it may be more difficult to train certain models such as neural networks which require large amounts of data. This is already a medium-sized dataset (not that large) so removing 500 instances is not insignificant.  

It is clear that we must drop some columns such as "Churn Reason", which would immediately inform our algorithms whether a customer churned or not. 

Some of the columns we drop:
- Count: Every value is equal to 1
- Quarter: Every value is equal to Q3
- Country: Every value is equal to "United States"
- State: Every value is equal to "California"

Description to be updated

In [61]:
df.drop(columns=["Churn Category", "Churn Reason", "Customer Status", "Churn Value", "Churn Score", "Count", "Quarter", "Lat Long"], inplace=True, axis=1)

In [62]:
df.isnull().sum(axis=0)

Customer ID                          0
Satisfaction Score                   0
Churn Label                          0
CLTV                                 0
Referred a Friend                    0
Number of Referrals                  0
Tenure in Months                     0
Offer                                0
Phone Service                        0
Avg Monthly Long Distance Charges    0
Multiple Lines                       0
Internet Service                     0
Internet Type                        0
Avg Monthly GB Download              0
Online Security                      0
Online Backup                        0
Device Protection Plan               0
Premium Tech Support                 0
Streaming TV                         0
Streaming Movies                     0
Streaming Music                      0
Unlimited Data                       0
Contract                             0
Paperless Billing                    0
Payment Method                       0
Monthly Charge           

In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 44 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Customer ID                        7043 non-null   object 
 1   Satisfaction Score                 7043 non-null   int64  
 2   Churn Label                        7043 non-null   object 
 3   CLTV                               7043 non-null   int64  
 4   Referred a Friend                  7043 non-null   object 
 5   Number of Referrals                7043 non-null   int64  
 6   Tenure in Months                   7043 non-null   int64  
 7   Offer                              7043 non-null   object 
 8   Phone Service                      7043 non-null   object 
 9   Avg Monthly Long Distance Charges  7043 non-null   float64
 10  Multiple Lines                     7043 non-null   object 
 11  Internet Service                   7043 non-null   objec

### Fixing data types

#### Encoding binary values

Multiple binary formats are given by "Yes" and "No" which we must convert appropriately.

In [64]:
def convert_binary(columns: list):
    for column in columns:
        df[column] = df[column].eq('Yes').mul(1)

In [65]:
df.groupby("Phone Service")["Customer ID"].nunique()

Phone Service
No      682
Yes    6361
Name: Customer ID, dtype: int64

In [66]:
binary_columns = ["Referred a Friend", "Churn Label", "Under 30", "Senior Citizen", "Married", "Dependents", "Phone Service", "Multiple Lines"]
convert_binary(binary_columns)

#### One hot encoding for churn label

In [67]:
df.head()

,Customer ID,Satisfaction Score,Churn Label,CLTV,Referred a Friend,Number of Referrals,Tenure in Months,Offer,Phone Service,Avg Monthly Long Distance Charges,...,Zip Code,Latitude,Longitude,Gender,Age,Under 30,Senior Citizen,Married,Dependents,Number of Dependents
0,8779-QRDMV,3,1,5433,0,0,1,None,0,0.00,...,90022,34.023810,-118.156582,Male,78,0,1,0,0,0
1,7495-OOKFY,3,1,5302,1,1,8,Offer E,1,48.85,...,90063,34.044271,-118.185237,Female,74,0,1,1,1,1
2,1658-BYGOY,2,1,3179,0,0,18,Offer D,1,11.33,...,90065,34.108833,-118.229715,Male,71,0,1,0,1,3
3,4598-XLKNJ,2,1,5337,1,1,25,Offer C,1,19.76,...,90303,33.936291,-118.332639,Female,78,0,1,1,1,1
4,4846-WHAFZ,2,1,2793,1,1,37,Offer C,1,6.33,...,90602,33.972119,-118.020188,Female,80,0,1,1,1,1


In [68]:
df.drop(["Customer ID", "Satisfaction Score"], axis=1, inplace=True)

#### Saving the new data

We save the new data in a new file

In [69]:
save_file = "Telco_data_clean.csv"

save_path = DATA_PATH + save_file

df.to_csv(save_path)